In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../train_data.csv', sep = '|', error_bad_lines=False, index_col=False, dtype='unicode')
df.head()

,SubjectID,form_name,feature_name,feature_value,feature_unit,feature_delta
0,533,Demographic,Gender,F,NaN,0.0
1,533,Demographic,Age,65,NaN,0.0
2,533,Demographic,Race,White,NaN,0.0
3,533,ALSHX,onset_delta,-1023,NaN,0.0
4,533,ALSHX,diag_delta,-44,NaN,0.0


In [9]:
feature_names = df[["form_name", "feature_name"]].drop_duplicates()
feature_names.to_csv('../feature_names.csv', sep='|', index=False)

In [18]:
feature_values = df[["form_name", "feature_name", "feature_value"]].drop_duplicates()
feature_values[np.isnan(feature_values.feature_value.convert_objects(convert_numeric=True))]
feature_values.to_csv('../feature_values.csv', sep='|', index=False)

,form_name,feature_name,feature_value
0,Demographic,Gender,F
2,Demographic,Race,White
5,ALSHX,onset_site,Bulbar
214,Lab Test,Urine Protein,-
219,Lab Test,Bilirubin (Direct),-
234,Lab Test,Urine Amorphous Crystals,+
244,Lab Test,Urine Glucose,-
245,Lab Test,Normocytic,+
255,Lab Test,WAS PATIENT FASTING?,NO
322,Adverse Event,Gastrointestinal signs and symptoms,Dysphagia aggravated;Dysphagia;Gastrointestina...
